In [ ]:
import datetime

curr = datetime.datetime.now()
later = curr + datetime.timedelta(minutes = -5)
curr.isoformat(), later.isoformat()

('2021-02-03T18:47:57.972702', '2021-02-03T18:42:57.972702')

In [ ]:
import requests
MAX = 1000
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

curr = datetime.datetime.now(); curr = curr.isoformat()
later = curr + datetime.timedelta(minutes = -5); later = later.isoformat()

PARAMS = {
    "rcstart": curr.isoformat(),
    "rcend": later.isoformat(),
    "format": "json",
    "rcnamespace": "*",
    "rcprop": "title|sizes|flags|user|timestamp",
    "list": "recentchanges",
    "action": "query",
    "rclimit": MAX,
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()

RECENTCHANGES = DATA['query']['recentchanges']

print("current time: ", curr.isoformat(), "later time: ", later.isoformat(), "\n")

i = 0
for rc in RECENTCHANGES:
    print(i, str(rc['title']), str(rc['timestamp']))
    i+=1

In [ ]:
import datetime, time
from sseclient import SSEClient as EventSource
import json
from dateutil.parser import parse
url = 'https://stream.wikimedia.org/v2/stream/recentchange'

curr = datetime.datetime.now(); later = curr + datetime.timedelta(minutes = -5);  
curr = curr.isoformat(); later = later.isoformat()

for event in EventSource(url):
    if event.event == 'message':
        try:
            change = json.loads(event.data)
        except ValueError:
            pass
        else:
            # print('{user} edited {title}'.format(**change))
            # print(change['meta']['dt'])
            # timee = parse(change['meta']['dt']); timee.isoformat()
            # if(timee > later):
            #     print('{user} edited {title}'.format(**change))
            # else:
            #     break
            

In [ ]:
curr.replace()

datetime.datetime(2021, 2, 4, 16, 13, 29, 326041)

# Work

Links
* https://wikitech.wikimedia.org/wiki/Event_Platform/EventStreams#Python

In [ ]:
pip install sseclient
import datetime, time, json
from sseclient import SSEClient as EventSource
from dateutil.parser import parse

## TASK 01

### Domains Report

In [ ]:
url = 'https://stream.wikimedia.org/v2/stream/revision-create'

while(True):
    count = 0
    dictt = {}
    curr = datetime.datetime.now();
    for event in EventSource(url):
        if event.event == 'message':
            try:
                change = json.loads(event.data)
            except ValueError:
                pass
            else:
                timee = parse(change['meta']['dt']); timee.isoformat()
                timee = timee.replace(tzinfo = None)
                if divmod(abs((curr - timee).total_seconds()), 60)[0] < float(1):
                    count+=1
                    if change['meta']['domain'] in dictt.keys():
                        dictt[change['meta']['domain']] += 1
                    else:
                        dictt[change['meta']['domain']] = 1
                else:
                    break
    print("\ntotal changes count: ", count)
    
    sorted_dict = {}
    sorted_keys = sorted(dictt, key=dictt.get, reverse=True)
    for w in sorted_keys:
        sorted_dict[w] = dictt[w]

    for key in sorted_dict:
        print(key, ": ", sorted_dict[key], " pages updated")


### Users Report

In [ ]:
url = 'https://stream.wikimedia.org/v2/stream/revision-create'
count = 0
while(True):
    dictt = {}
    curr = datetime.datetime.now();
    for event in EventSource(url):
        if event.event == 'message':
            try:
                change = json.loads(event.data)
            except ValueError:
                pass
            else:
                try:
                    if change['meta']['domain'] != "en.wikipedia.org" or change['performer']['user_is_bot']:
                        continue
                    timee = parse(change['meta']['dt']); timee.isoformat()
                    timee = timee.replace(tzinfo = None)
                    if divmod(abs((curr - timee).total_seconds()), 10)[0] < float(1):
                        if change['performer']['user_text'] in dictt.keys():
                            max(dictt[change['performer']['user_text']], change['performer']['user_edit_count'])
                        else:
                            dictt[change['performer']['user_text']] = change['performer']['user_edit_count']
                    else:
                        break
                except KeyError:
                    dictt[change['performer']['user_text']] = -1
                    print("anonymous IP address received")

    sorted_dict = {}
    sorted_keys = sorted(dictt, key=dictt.get, reverse=True)
    for w in sorted_keys:
        sorted_dict[w] = dictt[w]
    print(count, ' minute')
    count+=1
    for key in sorted_dict:
        print(key, ": ", sorted_dict[key])


## Bonus Task